In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
import torch.backends.cudnn as cudnn
import torch
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [4]:
# !pip install torch
# !pip install natsort
# !pip install torchvision
# !pip install pyyaml
# !pip install pandas
# !pip install nltk

In [5]:
cudnn.benchmark = True
cudnn.deterministic = False

In [6]:
torch.cuda.is_available()

True

In [7]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [8]:
opt = get_config("config_files/en_filtered_config.yaml")
train(opt, amp=False)
%tb

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data/training
opt.select_data: ['']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data/training	 dataset: 
all_data/training/training
sub-directory:	/training	 num samples: 1536
num total samples of : 1536 x 1.0 (total_data_usage_ratio) = 1536
num samples of  per batch: 64 x 1.0 (batch_ratio) = 64
--------------------------------------------------------------------------------
Total_batch_size: 64 = 64
--------------------------------------------------------------------------------
dataset_root:    all_data/validation	 dataset: /
all_data/validation/validation
sub-directory:	/validation	 num samples: 263
-------------------------------------------------------------------

RuntimeError: DataLoader worker (pid(s) 9400, 12864, 3924, 12784, 11544) exited unexpectedly